Radares Meteorológicos

🧭 Introducción
Este cuaderno enseña cómo acceder, procesar y visualizar datos de radares meteorológicos del IDEAM desde AWS.

📚 Descripción general
En este cuaderno aprenderás a acceder y procesar datos de radar meteorológico en Colombia. Explorarás cómo leer archivos en formato RAWDSX2 utilizando la biblioteca raw2zarr, convertir los datos a formatos cloud-nativos (Zarr), y realizar visualizaciones básicas de reflectividad y otros productos radar.

Los datos provienen de la red de radares del IDEAM (Instituto de Hidrología, Meteorología y Estudios Ambientales de Colombia), que monitorea la atmósfera en tiempo real para pronóstico de tiempo severo y estimación de precipitación.

In [ ]:
# Librerías necesarias
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmweather
import fsspec
import matplotlib.pyplot as plt
import pandas as pd
import xradar as xd
from cartopy import geodesic
from shapely.geometry import Point

1. 🌎 Red de Radares del IDEAM en AWS
Colombia cuenta con una red de radares meteorológicos operados por diferentes instituciones. El IDEAM publica datos de 4 radares en banda C mediante un bucket público en Amazon Web Services (AWS).

Radar

Ubicación

Banda

Alcance

Carimagua

Meta

C

~200 km

Guaviare

Guaviare

C

~200 km

Munchique

Cauca

C

~200 km

Barrancabermeja

Santander

C

~200 km

1.1 🗺️ Ubicación de radares del IDEAM
El siguiente mapa muestra la ubicación geográfica de los radares IDEAM disponibles en AWS:

In [ ]:
# Cargar los datos
df_radares = pd.read_csv("../data/radar_locations.csv")

# Crear figura con proyección geográfica
fig, ax = plt.subplots(
    figsize=(8, 8),
    subplot_kw={"projection": ccrs.PlateCarree()},
    dpi=120
)

# Configurar mapa base
ax.set_extent([-85, -65, -5, 15])
ax.add_feature(cfeature.LAND, facecolor="whitesmoke")
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.BORDERS, linestyle=":")
ax.gridlines(draw_labels=True, linestyle="--", linewidth=0.5)

# Dibujar cada radar
for _, row in df_radares.iterrows():
    lon, lat = row["lon"], row["lat"]
    name = row["Name"]
    color = row["color"]
    
    # Punto del radar
    ax.plot(lon, lat, "o", color=color, markersize=6)
    ax.text(lon + 0.3, lat, name, fontsize=8)

    # Determinar radio en km (IDIGER tiene menor alcance)
    radio_km = 40 if "IDIGER" in name.upper() else 200
    
    # Dibujar círculo geodésico
    circle = geodesic.Geodesic().circle(lon=lon, lat=lat, radius=radio_km * 1000, n_samples=100)
    ax.plot(*circle.T, color=color, linewidth=1, alpha=0.3)

ax.set_title("Red de radares meteorológicos de Colombia")
plt.tight_layout()

2.2 📡 Radares meteorológicos del IDEAM disponibles en AWS
El Instituto de Hidrología, Meteorología y Estudios Ambientales (IDEAM) ha puesto a disposición pública los datos de sus radares meteorológicos mediante un repositorio en la nube de Amazon Web Services (AWS). Este repositorio facilita el acceso a información cruda de reflectividad y otros productos en formato propietario (SIGMET/IRIS), permitiendo su análisis mediante herramientas especializadas.

La estructura del bucket es la siguiente:

s3://s3-radaresideam/l2_data/YYYY/MM/DD/Radar_name/RRRAAMMDDTTTTTT.RAWXXXX 

Donde:

YYYY, MM, DD indican el año, mes y día del escaneo.

Radar_name es el nombre del radar (por ejemplo: Guaviare, Munchique, Barrancabermeja, Carimagua).

RRRAAMMDDTTTTTT.RAWXXXX es el nombre del archivo en formato SIGMET, compuesto por:

RRR: código del radar (ej. GUA para Guaviare)

AA: año en 2 dígitos

MM: mes

DD: día

TTTTTT: hora de adquisición (UTC)

RAWXXXX: identificador interno generado por el software IRIS

IDEAM actualmente dispone de archivos para sus radares principales desde aproximadamente 2018. Los datos están organizados por fecha, por lo que es necesario navegar dentro del bucket por año, mes y día para acceder a los archivos específicos.

💡 Tip: En los notebooks también puedes ejecutar comandos de línea de comandos (CLI) anteponiendo un signo de exclamación (!).

Por ejemplo, para listar los archivos disponibles en el bucket de radares del IDEAM para una fecha específica:

!aws s3 ls --no-sign-request s3://s3-radaresideam/l2_data/2021/09/19/

In [ ]:
!aws s3 ls --no-sign-request s3://s3-radaresideam/l2_data/2021/09/19/

3. 📥 Acceso a los datos de radar usando Python
IDEAM publica los datos de sus radares meteorológicos en un bucket público de Amazon Web Services (AWS). Gracias a herramientas como fsspec y xradar, es posible explorar estos datos, filtrarlos por fecha y estación, y analizarlos sin necesidad de descargarlos manualmente desde la web.

3.1 🔎 Exploración de archivos disponibles en el bucket
Primero, se establece una conexión con el bucket S3 de IDEAM utilizando fsspec. Esto permite buscar archivos según una ruta de interés que incluye fecha y nombre del radar.

In [ ]:
import fsspec

# Crear sistema de archivos con acceso anónimo
fs = fsspec.filesystem("s3", anon=True)

# Buscar archivos de radar disponibles en una fecha específica
files = sorted(fs.glob("s3-radaresideam/l2_data/2022/08/09/Carimagua/CAR22080919*"))
files[:5]  # Mostrar los primeros 5 archivos encontrados

🛠️ Personalización de la búsqueda

La ruta utilizada en el método fs.glob():

s3-radaresideam/l2_data/2022/08/09/Carimagua/CAR22080919*

puede ser modificada para consultar archivos de otras fechas o radares. Solo debes ajustar los siguientes componentes:

2022/08/09: la fecha del escaneo en formato YYYY/MM/DD

Carimagua: el nombre del radar (por ejemplo: Guaviare, Munchique, Barrancabermeja)

CAR22080919*: el patrón del nombre del archivo. Puedes usar comodines como * para seleccionar varios archivos.

Esto te permite explorar diferentes días o estaciones según tu interés de análisis.

3.2 📡 Lectura de archivos mediante streaming
Una vez localizado un archivo de interés, podemos leerlo directamente desde AWS mediante streaming sin necesidad de descargarlo al sistema local. Utilizamos fsspec.open() para abrir el archivo en modo lectura y luego xradar lo lee directamente, retornando una estructura jerárquica del tipo DataTree que organiza los barridos (sweeps) contenidos en el archivo.

In [ ]:
# Seleccionamos un archivo del bucket
filepath = f"s3://{files[7]}"

# Opciones de acceso a S3 (anónimo)
storage_options = {"anon": True}

# Abrimos el archivo en modo streaming y lo leemos con xradar
stream = fsspec.open(filepath, mode="rb", **storage_options).open()
radar = xd.io.open_iris_datatree(stream.read())

# Visualizamos el contenido del datatree
display(radar)

💡 Ventaja del streaming: Los datos se leen directamente desde AWS sin ocupar espacio en disco. Esto es especialmente útil cuando trabajas con múltiples archivos o volúmenes grandes de datos.

📘 xradar utiliza xarray y devuelve una estructura tipo DataTree, que organiza jerárquicamente la información contenida en los archivos SIGMET/IRIS, incluyendo múltiples niveles de escaneo por elevación.

3.3 📊 Gráfico de reflectividad
Una vez cargado el archivo con xradar, podemos acceder a un barrido individual (por ejemplo, sweep_0) y visualizar la reflectividad horizontal (DBZH) utilizando la funcionalidad xarray.plot que se encuentra incorporada en xarray.

Este primer gráfico mostrará la reflectividad en coordenadas de azimut y rango, tal como fue registrada por el radar.

In [ ]:
# Accedemos al primer barrido (elevación más baja)
sweep = radar["sweep_0"]

# Visualizamos la reflectividad horizontal (DBZH)
sweep["DBZH"].plot(cmap="ChaseSpectral", vmin=-10, vmax=60);

🎯 Este tipo de visualización es útil para inspeccionar rápidamente los valores de reflectividad en coordenadas polares. La reflectividad se mide en decibelios Z (dBZ) y representa la intensidad de los retornos del radar.

3.4 🌐 Georreferenciación de datos de radar
Los datos de radar inicialmente se encuentran en coordenadas polares: azimuth (ángulo de rotación horizontal) y range (distancia radial). Para representar estos datos en un sistema de coordenadas cartesianas (x, y, z), es necesario aplicar un proceso de georreferenciación.

La librería xradar permite realizar este proceso fácilmente con el método .xradar.georeference().

In [ ]:
# Aplicamos georreferenciación al datatree completo
radar = radar.xradar.georeference()

# Inspeccionamos el barrido ya georreferenciado
display(radar["sweep_0"])

🌐 La georreferenciación agrega nuevas coordenadas físicas x, y y z que representan la ubicación espacial real del haz de radar sobre la superficie terrestre. Esto es esencial para graficar en mapas o combinar con otras fuentes geoespaciales.

3.5 🗺️ Visualización georreferenciada
Una vez que los datos han sido georreferenciados, podemos generar una visualización en coordenadas cartesianas usando las nuevas variables x e y. Esto permite observar la reflectividad del radar en el espacio físico real, facilitando su integración con mapas o capas geográficas.

In [ ]:

# Graficamos la reflectividad horizontal usando coordenadas georreferenciadas
radar["sweep_0"]["DBZH"].plot(
    x="x",
    y="y",
    cmap="ChaseSpectral",
    vmin=-10,
    vmax=60
);

🗺️ Esta visualización representa los datos del radar en un plano cartesiano, donde la ubicación espacial ya no depende de azimut y rango, sino de distancias en metros respecto al radar (coordenadas x, y).

3.6 🗺️ Visualización en sistema de coordenadas geográficas
Para representar los datos de reflectividad en un mapa geográfico, es necesario convertir el sistema de coordenadas cartesianas del radar a un sistema proyectado. Podemos obtener esta proyección directamente desde el objeto radar utilizando xradar.georeference.get_crs() y crear una proyección compatible con cartopy.

In [ ]:
# Obtener el sistema de referencia proyectado desde el radar
proj_crs = xd.georeference.get_crs(radar["sweep_0"].ds)

# Crear una proyección de Cartopy usando ese CRS
cart_crs = ccrs.Projection(proj_crs)

🧭 Esta proyección permitirá superponer correctamente los datos del radar sobre mapas de costas, fronteras y otros elementos geográficos usando cartopy.

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
radar["sweep_0"]["DBZH"].plot(
    x="x",
    y="y",
    cmap="ChaseSpectral",
    transform=cart_crs,
    cbar_kwargs=dict(pad=0.075, shrink=0.75),
    vmin=-10,
    vmax=60,
)
ax.coastlines()
ax.gridlines(draw_labels=True, ls="--", lw=0.5);🌐 Este gráfico muestra la reflectividad del radar proyectada sobre un sistema de coordenadas geográficas. Es útil para análisis meteorológico, estudios hidrológicos y validación con otras fuentes de datos espaciales.

4. 🌧️ Estimación de lluvia a partir de reflectividad
Una de las aplicaciones más comunes del radar meteorológico es la estimación de tasas de precipitación. Para ello se utiliza una relación empírica entre la reflectividad Z y la intensidad de lluvia R, conocida como relación Z–R.

Una de las formas más conocidas y utilizadas es la propuesta por Marshall & Gunn (1953):

Z
=
200
⋅
R
1.6
Z=200⋅R 
1.6
 
Despejando para 
R
R:

R
=
(
Z
200
)
1
/
1.6
R=( 
200
Z
​
 ) 
1/1.6
 


⚠️ Conversión de unidades

La reflectividad del radar está dada en unidades decibélicas 
Z
d
B
Z
Z 
dBZ
​
 , relacionadas con la reflectividad lineal 
Z
Z mediante:

Z
d
B
Z
=
10
⋅
log
⁡
10
(
Z
)
Z 
dBZ
​
 =10⋅log 
10
​
 (Z)
Para aplicar la ecuación Z-R, primero debemos convertir a unidades lineales:

Z
=
1
0
(
Z
d
B
Z
/
10
)
Z=10 
(Z 
dBZ
​
 /10)
 


A continuación, implementamos este procedimiento paso a paso para estimar la lluvia a partir de un barrido de reflectividad horizontal (DBZH):

In [ ]:
# Reflectividad en unidades logarítmicas (dBZ)
ref_log = radar["sweep_0"]["DBZH"]

# Conversión a unidades lineales
ref_lin = 10 ** (ref_log / 10)

# Aplicación de la ecuación de Marshall & Gunn (1953)
rr = (1 / 200) ** (1 / 1.6) * ref_lin ** (1 / 1.6)

Ahora podemos visualizar el campo de lluvia generado

4.1 🗺️ Visualización del campo de lluvia